# Modelling atmospheric radiative transfer

<span style="color:darkgray">**Robin J Hogan and Shannon L Mason**</span>

## 1. Introduction

In this practical, the offline version of ECMWF’s atmospheric radiation scheme *ecRad* [(Hogan and Bozzo, 2018)](https://agupubs.onlinelibrary.wiley.com/doi/pdfdirect/10.1029/2018MS001364)
will be used to gain an understanding of how radiative transfer is modelled, and how both radiative fluxes and atmospheric heating rates are sensitive to gas concentrations, clouds and aerosols. 

The input data consist of a slice through the atmosphere from the North Pole to the South Pole at a longitude of 5$^{\circ}$E at 12 UTC on 11 July 2019, as represented by ECMWF’s ERA5 reanalysis system. 
The data are stored at 137 levels from the surface to the middle mesosphere (a pressure of 1 Pa), and have been extracted at every 0.5$^{\circ}$ interval in latitude. 
This slice includes a range of atmospheric and surface conditions, from marine stratocumulus to deep convection, sea ice to a hot desert surface, and pristine marine air over the Southern Ocean to the dust-laden Saharan boundary layer.

<span style="color:red">**Visualise the input data along the slice by running the `plot_input` script.**</span>

***Note: we run the plotting scripts as Python modules, but they can also be run in the command-line if you wish.***

In [ ]:
#First we import all of our plotting scripts
import plot_input, plot_input_profile, plot_output, plot_output_profile, plot_output_scalar, compare_output, compare_output_profile, compare_output_scalar

In [ ]:
plot_input.main("era5slice.nc")

Profiles of input data at a given latitude can be plotted using 

    plot_input_profile.main(10, "era5slice.nc")

<span style="color:red">**Try plotting profiles of the input data at a few different latitudes by copying and changing the lines below:**</span>

## 2. The Control experiment

The contents of this directory will include the following files:

- **`radiation_practical.ipynb`** 
This notebook.

- **`ecrad_1.3.1/`**
The ecrad source code.

- **`ecrad`** 
A symbolic link to the *ecRad* executable.

- **`data/`** 
A symbolic link to the *ecRad* data directory.

- **`config.nam`** 
A Fortran namelist file for configuring *ecRad* in its control configuration.

- **`era5slice.nc`** 
A netCDF file containing the input data; some of its contents are shown in Section 1 above.

- **`*.py`** 
Python scripts for plotting the data; see Section 6 for a description of all the scripts.

- **`ecradplot/`** 
A directory containing the Python library for plotting *ecRad* data.

- **`clean.sh`**
A little script for cleaning up changes to the working directory, including reverting this Notebook back to a clean state (any changes you've made in the Notebook be coped with a timestamp).

### 2.1 Running *ecRad*

Now the following runs *ecRad* on the input data using the configuration specified in `config.nam`, and writes the output to a NetCDF file `control.nc`:

    ./ecrad config.nam era5slice.nc control.nc
    
***NOTE: This can be run from a terminal within Jupyter Lab (select "terminal" from the "new launcher" button), or within a cell in this Notebook by putting the "!" character at the front of the line.***

### 2.2 Visualising *ecRad* outputs

#### 2.2.1 Plotting 2D fields

<span style="color:red">**To visualise some key fields from the output file `control.nc`, run the `plot_output.py` script with:**</span>
     
    plot_output.main("era5slice.nc", "control.nc")

#### 2.2.2 Plotting profiles

<span style="color:red">**Plot profiles of *ecRad* output at a specific latitude (e.g. 10N) by running `plot_output_profile.py`**</span>

    plot_output_profile.main(10, "era5slice.nc", ["control.nc"])

where `latitude` is a number in the range [-90,90]; e.g. 10, producing a file `era5slice_control_profile_10N.png`. 

<span style="color:red">**Try plotting a few different radiation profiles at the same locations you looked at above.**</span>

#### 2.2.3 Plotting 1D fields

<span style="color:red">**Run `plot_output_scalar` to plots the surface and top-of-atmosphere fluxes along the slice:**</span>

    plot_output_scalar.main("era5slice.nc", ["control.nc"])

producing the file `era5slice_control_surface_and_TOA.png`

***NOTE: if additional ecRad output files from other experiments (see later sections) are provided as arguments to `plot_output_scalar`, they will be overplotted in different colours.***

Have a look at the figures that have been created. There are three types of variable:

- Upwelling and downwelling shortwave (solar) and longwave (thermal-infrared) irradiances, i.e. the radiativepower passing through a horizontal surface at a particular height in the atmosphere, with units of W m$^{−2}$. 
The labels of the plots use the more common term "flux" instead of irradiance, and we will use the same for the remainder of this document. 
The net flux is defined as the downwelling minus the upwelling flux.

- Shortwave and longwave atmospheric heating rates (HR) computed from the vertical divergence of the netflux in pressure coordinates: $$\mathrm{HR} = −\dfrac{g_0}{C_p} \dfrac{\mathrm{d} F_n}{\mathrm{d}p},$$ where $F_n$ is the net flux (shortwave or longwave), $p$ is pressure, $g_0$ is the acceleration due to gravity (standard gravity) and $C_p$ is the specific heat of dry air. 
The plots use units of Kelvin per day (K d$^{−1}$).

- Cloud radiative effect, which is a flux minus the equivalent flux computed without clouds present in the profile, and is commonly used to quantify the warming or cooling effect of clouds on the climate system. *ecRad* output files contain both all-sky and clear-sky fluxes.

### <span style="color:red">2.3 Questions</span>

#### <span style="color:red">1. As anyone who has flown in an aeroplane knows, clouds (especially low-level clouds) are turbulent. Explain why, with reference to the radiation fields. Thick low-level clouds are present at 71$^\circ$N and 15$^\circ$S.</span>

#### <span style="color:red">2. The net (shortwave plus longwave) heating rate at the stratopause (around 1 hPa) in the Tropics is around +10 K d$^{−1}$. Explain why you would not expect the temperature here a week later to be much warmer.</span>

#### <span style="color:red">3. Explain why the shortwave cloud radiative effect drops sharply polewards of around 80$^\circ$N, yet optically thick clouds are present all the way to the North Pole.</span>

## 3. Greenhouse gases

The remainder of the practical consists of running *ecRad* with different configurations, plotting the change to fluxes and heating rates, and trying to understand the differences. This section is concerned with investigating the effect of changing the concentration of absorbing gases, but the general procedure for performing an experiment is the same in the following two sections which concern perturbing clouds and aerosols.

### 3.1 Doubled-CO2 experiment

Let’s investigate the radiative impact of doubling the concentration of carbon dioxide. Usually in the context of climate change research, a doubled-CO2 experiment is with respect to preindustrial concentrations (surface volume mixing ratio of around 280 ppmv), but since our control experiment is from 2019 (surface volume mixing ratio ofaround 412 ppmv) we will double the 2019 value. 

<span style="color:red">**Make a new copy of the configuration file with for this experiment (`config_co2x2.name`) using the Jupyter file browser, or in the command-line, e.g.**</span>

``cp config.nam config_co2x2.nam``

<span style="color:red">**and edit the new configuration file with the Jupyter editor to change these lines:**</span>

    !co2_scaling = 1.0,
    experiment_name = "Control",

<span style="color:red">**to these:**</span>

    co2_scaling = 2.0,
    experiment_name = "2x CO2",

<span style="color:red">**Then run *ecRad* on it, giving `co2x2.nc` as the filename for the output:**</span>

``./ecrad config_co2x2.nam era5slice.nc co2x2.nc``

For a full list of available namelist parameters, see the User Guide at the [*ecRad* web site](https://confluence.ecmwf.int/display/ECRAD). 
In the namelist, lines starting with the `!` character are ignored.

You can compare this experiment with the control as follows:

    compare_output.main("era5slice.nc", "control.nc", "co2x2.nc")

and plot selected profiles at a latitude of 17$^{\circ}$S with

    compare_output_profile.main(-17, "era5slice.nc", "control.nc", ["co2x2.nc"])

***Note that if multiple output files are provided as arguments then they will be overplotted, each using the same control.*** 

To compare the fluxes and cloud radiative effect at the surface and top-of-atmosphere at all latitudes, use

    compare_output_scalar.main("era5slice.nc", "control.nc", ["co2x2.nc"])

where multiple output files can be provided as arguments.

### <span style="color:red">3.2 Questions and exercises </span>

#### <span style="color:red">1.  Will the stratosphere warm or cool in response to increasing CO2 and why?</span>

#### <span style="color:red">2.  By  perturbing  ozone  concentrations  (e.g.  setting  the  namelist  variable `o3_scaling` to `0.7`),  explain  the mechanisms by which stratospheric ozone depletion leads to a cooling of the Earth system in the shortwave and the longwave.</span>

#### <span style="color:red">3. By perturbing methane concentrations (e.g. setting the namelist variable `ch4_scaling` to `2.0`), estimate how many times stronger a greenhouse gas methane is than carbon dioxide, per molecule, in terms of instantaneous longwave radiative forcing at top-of-atmosphere. You need to know that the 2019 concentration of methane is around 1.905 ppmv, and that the number of molecules of an ideal gas is proportional its volume mixing ratio with no dependence on the molecular mass.  The 5th IPCC report stated that methane has a global warming potential 28 times that of carbon dioxide over a 100-year time period.  Why might this be different from your number?</span>

## 4. Clouds 

Some  of  the  following  involve  changing  the  algorithm  used  to  perform  the  radiative  transfer.  The  impact  on computational cost (important for the configuration of a radiation scheme in a weather or climate model) can be estimated by prefixing the ``./ecrad`` command with `time` and looking at the `user` time reported after *ecRad* has finished.

### <span style="color:red">4.1 Questions and exercises</span>
    
#### <span style="color:red">1.  Many   models   neglect   scattering   of   longwave   radiation   by   clouds   for   computational   expediency.  What  is  the  change  to  longwave  cloud  radiative  effect  when  longwave  scattering  is  turned  off  (set `do_lw_cloud_scattering` to `false`)?  Why is it of this sign?</span>

#### <span style="color:red">2.  The control simulation assumes sub-grid fluctuations in cloud water content such that the fractional standard deviation (the standard deviation divided by the mean) is one. What is the impact on shortwave and longwave cloud radiative effect when clouds are homogeneous (set `fractional_std` to `0.0`) and why?</span>

#### <span style="color:red">3.  The  control  simulation  makes  the  assumption  of exponential-random vertical  overlap  of  clouds,  which means  that  vertically  separated  clouds  are  randomly  overlapped  with  respect  to  each  other,  while  vertically  contiguous  cloud  layers  decorrelate  with  separation  distance  according  to  an  overlap  decorrelation length that varies with latitude. What is the impact of making the clouds in all layers overlap randomly (set `overlap_decorr_length_scaling` to `0.0`) and why?</span>

#### <span style="color:red">4.  The  control  simulation  uses  the *Tripleclouds* solver  [(Shonk  and  Hogan,  2008)](https://git.ecmwf.int/projects/ECRAD/repos/ecrad/raw/practical/ecrad_practical.pdf?at=refs%2Fheads%2Fgithub#cite.Shonk+2008)  for  performing  radiative  transfer  calculations  in  the  presence  of  clouds,  including  the  effect  of  sub-grid  fluctuations  incloud  water  content. The  operational  ECMWF  model  uses  the *McICA* solver  [(Pincus  et  al.,  2003)](https://git.ecmwf.int/projects/ECRAD/repos/ecrad/raw/practical/ecrad_practical.pdf?t=refs%2Fheads%2Fgithub#cite.Pincus+2003), which uses a stochastic model to generate sub-grid cloud distributions and can be selected by changing `sw_solver_name` and `lw_solver_name` both to `McICA`.  Looking at the atmospheric heating rates (e.g.using `plot_output_profile` at 7$^{\circ}$N), and the overall computational cost, explain the pros and cons of using *McICA* versus *Tripleclouds* in an operational context.</span>

#### <span style="color:red">5.  Ice optical properties are an uncertain part of a radiative transfer scheme.  Investigate the size of this uncertainty by switching from the default Fu ice optics model to that of [Yi et al. (2013)](https://git.ecmwf.int/projects/ECRAD/repos/ecrad/raw/practical/ecrad_practical.pdf?at=refs%2Fheads%2Fgithub#cite.Yi+2013) (set `ice_model_name` to `Yi`). Is the main impact in the longwave or shortwave?</span>

#### <span style="color:red">6. A unique property of *ecRad* is the availability of the *SPARTACUS* solver  [(Hogan  et  al.,  2016)](https://git.ecmwf.int/projects/ECRAD/repos/ecrad/raw/practical/ecrad_practical.pdf?at=refs%2Fheads%2Fgithub#cite.Hogan+2016)  for  approximately representing the 3D radiative effects of clouds, available by changing `sw_solver_name` and `lw_solver_name` both to `SPARTACUS`. What is the effect on both the radiation fields and the computational cost?</span>

## 5. Aerosols

### <span style="color:red">5.1 Questions and exercises</span>

#### <span style="color:red">1.  Turn  off  the  interaction  of  radiation  with  aerosols  by  setting  the  namelist  parameter `use_aerosols` to `false`. With reference to the surface fluxes, what do you think the impact of Saharan aerosols is on both daytime and nighttime surface temperature?</span>

#### <span style="color:red">2. The direct downwelling solar radiation is that part of the solar radiation field that has not yet been scattered, and to a first approximation its value at a particular point in the atmosphere is proportional to $\exp(−\tau)$, where $\tau$ is the optical depth of the atmosphere above that point along a path towards the sun.  Use a clear-sky profile of direct downwelling solar radiation over Africa to estimate the optical depth of the atmosphere with and without aerosols.</span>

#### <span style="color:red">3. Concentrating solar power installations focus the direct solar beam to generate energy, whereas photovoltaic power installations can use all downwelling solar radiation. Which of these installations is most sensitive to dust storms?</span>

#### <span style="color:red">4.  From the effect of aerosols on shortwave radiation at the top-of-atmosphere, do aerosols have a warming or cooling effect on climate?  What would the effect on climate be if you turned all the aerosols into black carbon (set each of the elements of `i_aerosol_type_map` to `11` and check the output of *ecRad*)? Why?</span>

## 6. Quick guide to Python plotting scripts

**``plot_input.main(<in.nc>)``** 
Plot the main fields in the input file (`era5slice.nc` in this practical) to the image file `<in>.png`. 

**``plot_input_profile.main(<lat>,<in.nc>)``** 
Plot profiles of a selection of properties in the input file at the specified latitude (in degrees north) to `<in>_profile_<lat>.png`. 

**``plot_output.main(<in.nc>,<out.nc>)``**
Plot the fluxes and heating rates from the specified output file versus latitude and pressure, with the input file providing the temperature and latitude information, and save the result in `<in>_<out>.png`.

**``plot_output_profile.main(<lat>,<in.nc>,[<out1.nc>,<out2.nc>, ...])``**
Plot profiles of variables from one or more output files at the specified latitude, saving the result in `<in>_<out1>[_<out2>...]_profile_<lat>.png`. 

**``plot_output_scalar.main(<in.nc>,[<out1.nc>,<out2.nc>...])``**
Line plots versus latitude of upwelling fluxes at top-of-atmosphere, downwelling fluxes at the surface, top-of-atmosphere cloud radiative effect, cloud cover and surface direct shortwave flux, for one or more output files. The result is saved in `<in>_<out1>[_<out2>...]_surface_and_TOA.png`. 

**``compare_output.main(<in.nc>,<control.nc>,<out2.nc>)``**
Plot the difference between the fluxes and heating rates in second output file (the experiment) and the first (the control), saving the result in `<in>_<out2>_vs_<control>.png.` 

**``compare_output_profile.main(<lat>,<in.nc>,<control.nc>, [<out2.nc>,<out3.nc>...])``** 
Compare profiles at the specified latitude of the fluxes and heating rates in several output files, where the first is treated as the control, and write the output to `<in>_<out2>[_<out3>...]_vs_<control>_profile_<lat>.png`. 

**``compare_output_scalar.main(<in.nc>,<control.nc>,[<out2.nc>,<out3.nc>...])``**
Compare net fluxes at top-of-atmosphere and the surface, as well as the cloud cover and surface direct shortwave flux, from several output files, where the first is treated as the control. The result is saved in `<in>_<out2>[_<out3>...]_vs_<control>_scalar.png`. 

# References

Hogan, R. J., and A. Bozzo, 2018: A flexible radiation scheme for the ECMWF model. *J. Adv. Model. Earth Syst.*, **10**, [doi:10.1029/2018MS001364](doi:10.1029/2018MS001364).

Hogan, R. J., S. A. K. Schäfer, C. Klinger, J.-C. Chiu and B. Mayer, 2016: Representing 3D cloud-radiation effects in two-stream schemes: 2. Matrix formulation and broadband evaluation. *J. Geophys. Res.*, **121**, 8583–8599.

Pincus, R., H. W. Barker, and J.-J. Morcrette, 2003: A fast, flexible, approximate technique for computing radiative transfer in inhomogeneous clouds. *J. Geophys. Res. Atmos.*, **108**, 4376, [doi:10.1029/2002JD003322](doi:10.1029/2002JD003322).

Shonk, J. K. P., and R. J. Hogan, 2008: Tripleclouds: an efficient method for representing horizontal cloud inho-mogeneity in 1D radiation schemes by using three regions at each height. *J. Climate*, **21**, 2352–2370.

Yi, B., P. Yang, B. A. Baum, T. L’Ecuyer, L. Oreopoulos, E. J. Mlawer, A. J. Heymsfield and K.-K. Liou, 2013: Influence of ice particle surface roughening on the global cloud radiative effect. *J. Atmos. Sci.*, **70**, 2794–2807.